In [1]:
import py2neo
import pandas as pd
import numpy as np
import math

In [2]:
from py2neo import Graph, Node, Relationship, NodeMatcher

In [3]:
def is_not_nan(value):
    if type(value) != str:
        if math.isnan(value):
            return False
        else:
            return True
    else:
        return True

In [4]:
is_not_nan(np.NaN)

False

In [5]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "password"))

In [6]:
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [7]:
students = pd.read_csv("McSQL Data - students.csv")
classes = pd.read_csv("McSQL Data - classes.csv")
countries = pd.read_csv("McSQL Data - countries.csv")
xtras = pd.read_csv("McSQL Data - extracurriculars.csv")

In [8]:
students.head()

,student_id,first_name,last_name,fav_class,xtra_id_1,xtra_id_2,xtra_id_3,fav_vacation_id,dream_vacation_id
0,hmf9kx,Hayden,French,CS3102,1,3,NaN,CRI,FJI
1,kgw4wm,Katlyn,Walter,ECON4220,2,5,3.0,GRC,JPN
2,stf8saa,Stephanie,Fissel,ECON3030,3,4,NaN,CRI,ESP
3,sjh7yg,Silas,Hayes,STAT1602,0,3,NaN,ITA,NZL
4,mep6jj,Marina,Peebles,ECON3030,2,1,NaN,ESP,NZL


In [9]:
countries.head()

,country_id,name,capital,population,gdp_per_capita,timezone
0,CRI,Costa Rica,San José,5136440,12244,GMT-6
1,ITA,Italy,Rome,60360000,33228,GMT+2
2,NZL,New Zealand,Wellington,4971000,42084,GMT+12
3,GRC,Greece,Athens,10720000,19583,GMT+3
4,ESP,Spain,Madrid,46940000,29600,GMT+2


In [10]:
xtras.head()

,xtra_id,name,semester,director_fname,director_lname,application
0,0,SOTL,both,Max,Tank,False
1,1,Cavalier Marching Band,fall,Andrew,Koch,True
2,2,French House,both,Rachel,Geer,True
3,3,Forge,both,Andy,Page,True
4,4,SWS,both,Claire,Duffy,True


In [11]:
classes.head(5)

,class_id,class_name,professor_fname,professor_lname,prereqs_1,prereqs_2,credits,gpa
0,CS3102,Theory of Computation,Nathan,Brunelle,CS2102,CS2110,3,3.19
1,ECON4220,Intr Finance & Macroeconomics,Eric,Van Wincoop,ECON3020,NaN,3,3.75
2,ECON4170,"Econ: Risk, Uncertainty & Info",Marc,Santugini,ECON3010,NaN,3,3.66
3,MUEN3630,Chamber Ensemble,I-Jen,Fang,NaN,NaN,1,3.96
4,FREN3029,Language House Conversation,Katia,Bellal,NaN,NaN,1,3.72


In [12]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("CREATE (:STUDENT {student_id:'" + row['student_id'] + "', first_name:'" + row['first_name'] + "', last_name:'" + row['last_name'] + "', fav_class:'" + row['fav_class'] 
          + "', xtra_id_1:'" + str(row['xtra_id_1']) + "', xtra_id_2:'" + str(row['xtra_id_2']) + "', xtra_id_3:'" + str(row['xtra_id_3']) + "', fav_vacation_id:'" + 
          row['fav_vacation_id'] + "', dream_vacation_id:'" + row['dream_vacation_id'] + "', last_name:'" + row['last_name'] + "'})")

In [13]:
for i in range(len(classes)):
    row = classes.iloc[i]
    graph.run("CREATE (:CLASS {class_id:'" + row['class_id'] + "', class_name:'" + row['class_name'] + "', professor_lname:'" + row['professor_lname'] + "', professor_fname:'" 
          + row['professor_fname'] + "', prereqs_1:'" + str(row['prereqs_1']) + "', prereqs_2:'" + str(row['prereqs_2']) + "', credits:'" + str(row['credits']) + "', gpa:'" 
          + str(row['gpa']) + "'})")

In [14]:
for i in range(len(countries)):
    row = countries.iloc[i]
    graph.run("CREATE (:COUNTRY {country_id:'" + row['country_id'] + "', name:'" + row['name'] + "', capital:'" + row['capital'] + "', population:'" + str(row['population'])
              + "', gdp_per_capita:'" + str(row['gdp_per_capita']) + "', timezone:'" + row['timezone'] + "'})")

In [15]:
for i in range(len(xtras)):
    row = xtras.iloc[i]
    graph.run("CREATE (:EXTRACURRICULAR {xtra_id:'" + str(row['xtra_id']) + "', name:'" + row['name'] + "', semester:'" + row['semester'] + "', director_fname:'" 
              + row['director_fname'] + "', director_lname:'" + row['director_lname'] + "', application:'" + str(row['application']) + "'})")

In [16]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (c:CLASS {class_id:'" + row['fav_class'] + "'}) CREATE (s)-[:LIKES]->(c)")

In [17]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (c:COUNTRY {country_id:'" + row['fav_vacation_id'] + "'}) CREATE (s)-[:HAS_BEEN_TO]->(c)")

In [18]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (c:COUNTRY {country_id:'" + row['dream_vacation_id'] + "'}) CREATE (s)-[:WANTS_TO_GO_TO]->(c)")

In [19]:
for i in range(len(students)):
    row = students.iloc[i]
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (e:EXTRACURRICULAR {xtra_id:'" + str(row['xtra_id_1']) + "'}) CREATE (s)-[:PARTICIPATES_IN]->(e)")
    graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (e:EXTRACURRICULAR {xtra_id:'" + str(row['xtra_id_2']) + "'}) CREATE (s)-[:PARTICIPATES_IN]->(e)")
    if is_not_nan(row['xtra_id_3']):
        temp = int(row['xtra_id_3'])
        graph.run("MATCH (s:STUDENT {student_id:'" + row['student_id'] +"'}), (e:EXTRACURRICULAR {xtra_id:'" + str(temp) + "'}) CREATE (s)-[:PARTICIPATES_IN]->(e)")

In [20]:
for i in range(len(classes)):
    row = classes.iloc[i]
    if is_not_nan(row['prereqs_1']):
        graph.run("MATCH (c1:CLASS {class_id:'" + row['class_id'] +"'}), (c2:CLASS {class_id:'" + row['prereqs_1'] + "'}) CREATE (c1)<-[:IS_PREREQ_OF]-(c2)")
    if is_not_nan(row['prereqs_2']):
        graph.run("MATCH (c1:CLASS {class_id:'" + row['class_id'] +"'}), (c2:CLASS {class_id:'" + row['prereqs_2'] + "'}) CREATE (c1)<-[:IS_PREREQ_OF]-(c2)")

In [21]:
for i in range(len(students)):
    row = students.iloc[i]
    if is_not_nan(row['xtra_id_1']):
        length = np.random.choice(np.arange(1,3))
        graph.run("MATCH (n:STUDENT {first_name: '" + row.first_name +"'})-[r:PARTICIPATES_IN]->(x:EXTRACURRICULAR {name: '" + xtras.iloc[int(students.iloc[i].xtra_id_1)].loc['name'] + "'}) SET r.years = " + str(length))
    if is_not_nan(row['xtra_id_2']):
        length = np.random.choice(np.arange(1,3))
        graph.run("MATCH (n:STUDENT {first_name: '" + row.first_name +"'})-[r:PARTICIPATES_IN]->(x:EXTRACURRICULAR {name: '" + xtras.iloc[int(students.iloc[i].xtra_id_2)].loc['name'] + "'}) SET r.years = " + str(length))
    if is_not_nan(row['xtra_id_3']):
        length = np.random.choice(np.arange(1,3))
        graph.run("MATCH (n:STUDENT {first_name: '" + row.first_name +"'})-[r:PARTICIPATES_IN]->(x:EXTRACURRICULAR {name: '" + xtras.iloc[int(students.iloc[i].xtra_id_3)].loc['name'] + "'}) SET r.years = " + str(length))


In [22]:
for i in range(len(students)):
    row = students.iloc[i]
    if is_not_nan(row['xtra_id_1']):
        length = np.random.choice(np.arange(2008, 2019))
        graph.run("MATCH (n:STUDENT {first_name: '" + row.first_name +"'})-[r:HAS_BEEN_TO]->(c:COUNTRY {country_id: '" + row.fav_vacation_id + "'}) SET r.year = " + str(length))